In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import os
import sys
import math
import time
from datetime import datetime
import copy
import numpy as np
from scipy.spatial import distance_matrix
from docplex.mp.model import Model
import docplex.mp.conflict_refiner as cr
from docplex.util.environment import get_environment
from docplex.mp.progress import *

In [2]:
os.getcwd()

'C:\\Users\\Zhifeng Guo\\python_code\\paper3_code\\Benchmark'

In [3]:
#label_df1=pd.read_csv("bc_datasets/Soybean/soybean.classes.txt",header=None)
#label_df1=pd.read_csv("bc_datasets/Protein/protein.classes.txt",sep=" ",header=None)
#label_df1=pd.read_csv("bc_datasets/Iris/iris.classes.txt",sep=" ",header=None)
#label_df1=pd.read_csv("bc_datasets/Wine/wine.classes.txt",sep=" ",header=None)
#label_df1=pd.read_csv("bc_datasets/Ionosphere/ionosphere.classes.txt",sep=" ",header=None)
#label_df1=pd.read_csv("bc_datasets/Control/control.classes.txt",sep=" ",header=None)
#label_df1=pd.read_csv("bc_datasets/Balance/balance.classes.txt",sep=" ",header=None)
label_df1=pd.read_csv("bc_datasets/Yeast/yeast.classes.txt",sep=" ",header=None)

In [4]:
p=label_df1.value_counts().shape[0]
print(p)

10


In [5]:
#sampled_df1=pd.read_csv("bc_datasets/Soybean/soybean.data.txt",sep=" ",header=None)
#sampled_df1=pd.read_csv("bc_datasets/Protein/protein.data.txt",sep=" ",header=None)
#sampled_df1=pd.read_csv("bc_datasets/Iris/iris.data.txt",sep=" ",header=None)
#sampled_df1=pd.read_csv("bc_datasets/Wine/wine.data.txt",sep=" ",header=None)
#sampled_df1=pd.read_csv("bc_datasets/Ionosphere/ionosphere.data.txt",sep=" ",header=None)
#sampled_df1=pd.read_csv("bc_datasets/Control/control.data.txt",sep=" ",header=None)
#sampled_df1=pd.read_csv("bc_datasets/Balance/balance.data.txt",sep=" ",header=None)
sampled_df1=pd.read_csv("bc_datasets/Yeast/yeast.data.txt",sep=" ",header=None)

sampled_df1.columns = [f'X{i+1}' for i in range(len(sampled_df1.columns))]
sampled_df1=sampled_df1.dropna(axis=1, how='any')
df_X=sampled_df1.loc[:, sampled_df1.columns.str.contains('X')]

In [6]:
sampled_df1.head()

,X1,X2,X3,X4,X5,X6,X7,X8
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22


In [7]:
sampled_df1.shape

(1484, 8)

In [8]:
dfij=pd.DataFrame(distance_matrix(df_X.values, df_X.values), index=df_X.index, columns=df_X.index)
dij=dfij.to_numpy()
#dij = np.round(dij, decimals=2)

In [9]:
dij.shape

(1484, 1484)

In [10]:
#done
#Constraints_data=pd.read_csv("bc_datasets/Soybean/soybean1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Soybean/soybean2.txt",sep="\t",header=None)

#done
#Constraints_data=pd.read_csv("bc_datasets/Protein/protein1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Protein/protein2.txt",sep="\t",header=None)

#done
#Constraints_data=pd.read_csv("bc_datasets/Iris/iris1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Iris/iris2.txt",sep="\t",header=None)

#Constraints_data=pd.read_csv("bc_datasets/Wine/wine1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Wine/wine2.txt",sep="\t",header=None)

#Constraints_data=pd.read_csv("bc_datasets/Ionosphere/ionosphere1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Ionosphere/ionosphere2.txt",sep="\t",header=None)

#Constraints_data=pd.read_csv("bc_datasets/Control/control1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Control/control2.txt",sep="\t",header=None)

#Constraints_data=pd.read_csv("bc_datasets/Balance/balance1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Balance/balance2.txt",sep="\t",header=None)

Constraints_data=pd.read_csv("bc_datasets/Yeast/yeast1.txt",sep="\t",header=None)
#Constraints_data=pd.read_csv("bc_datasets/Yeast/yeast2.txt",sep="\t",header=None)


Constraints_data.columns = [f'f{i+1}' for i in range(len(Constraints_data.columns))]

Constraints_data.head()

,f1,f2,f3
0,952,641,-1
1,1070,1409,-1
2,1131,1093,-1
3,1401,821,-1
4,412,1450,-1


In [11]:
ML=Constraints_data[Constraints_data['f3']==1]
CL=Constraints_data[Constraints_data['f3']==-1]

In [12]:
ML.shape


(148, 3)

In [13]:
CL.shape

(89, 3)

In [14]:
CL.head()

,f1,f2,f3
0,952,641,-1
1,1070,1409,-1
2,1131,1093,-1
3,1401,821,-1
4,412,1450,-1


In [15]:
N=range(sampled_df1.shape[0])

In [16]:
def CPLEX_Model(name,N,ML,CL,p,dij):
    mdl=Model(name=name,log_output=True)
    #mdl=Model(name=name)

    #print(mdl.parameters.workmem())
    #print(mdl.parameters.mip.strategy.file())
    #mdl.parameters.mip.strategy.file=2
    #mdl.context.cplex_parameters.threads = 12
    #mdl.parameters.workmem=2048
    #mdl.parameters.parallel=1


    #set variables
    x=mdl.continuous_var_dict([(i,j) for i in N for j in N],lb=0.0, name="x")
    y=mdl.binary_var_dict([j for j in N],name="y")
    
    mdl.add_constraints(x[i, j] -y[j]<= 0 for i in N for j in N)
    mdl.add_constraint(mdl.sum(y[j] for j in N) == p)
    mdl.add_constraints(mdl.sum(x[i, j] for j in N) == 1 for i in N)
    

    mdl.add_constraints(x[row[1], j]+x[row[2], j] -1<= 0  for row in CL.itertuples() for j in N)
    #mdl.add_constraints(x[row[2], j]+x[row[1], j] -1<= 0  for row in CL.itertuples() for j in N)

    mdl.add_constraints(x[row[1], j]-x[row[2], j] == 0  for row in ML.itertuples() for j in N)
    #mdl.add_constraints(x[row[2], j]-x[row[1], j] == 0  for row in ML.itertuples() for j in N)
    
    mdl.minimize(mdl.sum(dij[i, j]*x[i,j] for i in N for j in N))
    mdl.parameters.benders.strategy = 3 # for benders decomposition
    sol_model = mdl.solve(clean_before_solve=True)
    
    solve_status = mdl.get_solve_status()
    print(solve_status)
    return x, y, sol_model

In [17]:
def get_y_values(y):
    Yj=[]
    for i in N:
        if y[i].solution_value==1:
            Yj.append(i)
    return Yj

In [18]:
#p=10
times=[]
objective_value=[]
print(p)
start2=datetime.now()
x, y, sol_model=CPLEX_Model(name="CPLEX_Model",N=N,ML=ML,CL=CL,p=p,dij=dij)
running_time=datetime.now()-start2
#cluster_points=get_y_values(y=y)
#pd.DataFrame(cluster_points, columns=['cluster_points']).to_csv('results/without_500_20_cluster_points_'+str(k)+'_.csv')
obj=sol_model.get_objective_value()
objective_value.append(obj)
times.append(running_time)
print(running_time)

10
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201804051
CPXPARAM_Benders_Strategy                        3
Aggregator has done 219632 substitutions...
Tried aggregator 2 times.
MIP Presolve eliminated 219780 rows and 0 columns.
Aggregator did 219632 substitutions.
Reduced MIP has 2116037 rows, 1984108 columns, and 6213508 nonzeros.
Reduced MIP has 1484 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 6.80 sec. (13496.82 ticks)
Found incumbent of value 478.359658 after 8.66 sec. (15778.87 ticks)
Tried aggregator 1 time.
Reduced MIP has 2116037 rows, 1984108 columns, and 6213508 nonzeros.
Reduced MIP has 1484 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.80 sec. (2093.64 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

Tried aggregator 1 time.
Reduced MIP has 1366 rows, 2731 columns, and 131065 nonze

In [19]:
obj

252.32023347313242

In [119]:
final_point= get_y_values(y)

In [120]:
final_point

[85, 107, 205, 243, 307, 367]

In [350]:
3600+28*60+5

5285